In [1]:
# Import packages
import plotly.express as px
import pandas as pd



In [2]:
# Read in the data
data = pd.read_csv("/Users/bg/Desktop/Data_Analyst /Combined 14-1048 Sales 23.24.csv")
data['Date'] = pd.to_datetime(data['Date'])

In [3]:
data = data.drop(columns='Cat No')
data

In [4]:
#Plot Data 
def plot_sales_volumes(df: pd.DataFrame,
                           y: str) -> None:
    """General function to plot the sales data."""

    fig = px.line(df, x='Date', y=y, labels='Date')
    fig.update_layout(template="simple_white", font=dict(size=18), title_text='Company Sales',
                      width=650, title_x=0.5, height=400)

    return fig.show()


# Plot the Sales data
plot_sales_volumes(df=data, y='Sales')

In [5]:
# Import packages
from scipy.stats import boxcox

# Make the data stationary
data['Sales_Boxcox'], lam = boxcox(data['Sales'])
data["Sales_diff"] = data["Sales_Boxcox"].diff()
data.dropna(inplace=True)

# Plot the stationary Sales data
plot_sales_volumes(df=data, y='Sales_diff')

In [8]:
# Import packages
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

# Plot acf and pacf
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5), dpi=80)
plot_acf(data['Sales_diff'])
plot_pacf(data['Sales_diff'], method='ywm')
ax1.tick_params(axis='both', labelsize=12)
ax2.tick_params(axis='both', labelsize=12)
plt.show()

In [6]:
# Import packages
from scipy.special import inv_boxcox
from statsmodels.tsa.arima.model import ARIMA

# Split train and test
train = data.iloc[:-int(len(data) * 0.2)]
test = data.iloc[-int(len(data) * 0.2):]

# Build ARIMA model
model = ARIMA(train['Sales_Boxcox'], order=(10, 1, 10),
              seasonal_order=(1, 1, 1, 12)).fit()
boxcox_forecasts = model.forecast(len(test))
forecasts = inv_boxcox(boxcox_forecasts, lam)

In [7]:
# Import packages
import plotly.graph_objects as go

def plot_forecasts(forecasts: list[float], title: str) -> None:
    """Function to plot the forecasts."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train['Date'], y=train['Sales'], name='Train'))
    fig.add_trace(go.Scatter(x=test['Date'], y=test['Sales'], name='Test'))
    fig.add_trace(go.Scatter(x=test['Date'], y=forecasts, name='Forecast'))
    fig.update_layout(template="simple_white", font=dict(size=18), title_text=title,
                      width=650, title_x=0.5, height=400, xaxis_title='Date',
                      yaxis_title='Sales')

    return fig.show()


# Plot the forecasts
plot_forecasts(forecasts, 'SARIMA')